In [1]:
from utils.loader import FactoryLoader
from utils.ml import MLPipeline
from utils.preprocessing import PreprocessingFactory
from utils.feature_extraction import *
from utils.utils import *

VAL_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val"
TRAIN_PATH = r"C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train"

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
percent = 100
random = False
batch_size = 24
th = 0.05

# Initialize the FactoryLoader
factory = PreprocessingFactory()
factory.gaussian_smoothing(5)
factory.clahe(clip_limit=1.5)
factory.pad2square(fill=np.nan)
factory.resize((150,150))
factory.hair_removal()
factory.normalize2float()

factory_loader = FactoryLoader(path=VAL_PATH, batch_size=batch_size, factory=factory, percentage=percent, shuffle=random)

# Create the feature extraction pipeline strategy and add desired features
strategy = FeatureExtractionStrategy()

strategy.add_extractor(GradientExtractor(threshold=th)) # Add gradient feature

strategy.add_extractor(ColorMomentsExtractor("rgb", threshold=th))   # Add color moments feature
strategy.add_extractor(ColorMomentsExtractor("lab", threshold=th))   # Add color moments feature
strategy.add_extractor(ColorMomentsExtractor("hsv", threshold=th))   # Add color moments feature

strategy.add_extractor(LBPExtractor(radius=1, n_points=8))
strategy.add_extractor(LBPExtractor(radius=2, n_points=16))
strategy.add_extractor(LBPExtractor(radius=3, n_points=24))
strategy.add_extractor(LBPExtractor(radius=4, n_points=32))

strategy.add_extractor(FourierTransformExtractor())

strategy.add_extractor(GLCMExtractor(properties=['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation',]))

pipeline = MLPipeline(dataset_path=TRAIN_PATH, preprocessing_factory=factory, feature_strategy=strategy, classifiers=[], percentage=percent, verbose=True, shuffle=random, batch_size=batch_size)

INFO:utils.ml:MLPipeline initialized with dataset path: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\train
INFO:utils.ml:Preprocessing steps


In [3]:
pipeline.run_feature_extraction()

INFO:utils.ml:Running feature extraction...


Processed 5/634 batches.
Processed 10/634 batches.
Processed 15/634 batches.
Processed 20/634 batches.
Processed 25/634 batches.
Processed 30/634 batches.
Processed 35/634 batches.
Processed 40/634 batches.
Processed 45/634 batches.
Processed 50/634 batches.
Processed 55/634 batches.
Processed 60/634 batches.
Processed 65/634 batches.
Processed 70/634 batches.
Processed 75/634 batches.
Processed 80/634 batches.
Processed 85/634 batches.
Processed 90/634 batches.
Processed 95/634 batches.
Processed 100/634 batches.
Processed 105/634 batches.
Processed 110/634 batches.
Processed 115/634 batches.
Processed 120/634 batches.
Processed 125/634 batches.
Processed 130/634 batches.
Processed 135/634 batches.
Processed 140/634 batches.
Processed 145/634 batches.
Processed 150/634 batches.
Processed 155/634 batches.
Processed 160/634 batches.
Processed 165/634 batches.
Processed 170/634 batches.
Processed 175/634 batches.
Processed 180/634 batches.
Processed 185/634 batches.
Processed 190/634 bat

INFO:utils.ml:Feature extraction completed. Extracted 15195 features.


Processed 634/634 batches.


In [4]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

rf1 = RandomForestClassifier(n_estimators=100)
rf2 = RandomForestClassifier(n_estimators=150)
rf3 = RandomForestClassifier(n_estimators=250)

rf4 = RandomForestClassifier(n_estimators=100, max_features='sqrt', random_state=42) # Random Forest with Feature Selection via max_features

rf5 = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=42) # Random Forest with Out-of-Bag Error (OOB)

rf6 = RandomForestClassifier(n_estimators=100, bootstrap=False, random_state=42) # Random Forest with Bootstrap Disabled

xgb1 = XGBClassifier(n_estimators=350)
xgb2 = XGBClassifier(n_estimators=450)
xgb3 = XGBClassifier(n_estimators=550)
xgb4 = XGBClassifier(learning_rate=0.05, n_estimators=400, max_depth=3, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb5 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=7, min_child_weight=3, subsample=0.7, colsample_bytree=0.7)
xgb6 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=5, min_child_weight=5, gamma=0.2, subsample=0.8, colsample_bytree=0.8)
xgb7 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=5, min_child_weight=1, subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1, reg_lambda=0.1)
xgb8 = XGBClassifier(learning_rate=0.05, n_estimators=500, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb9 = XGBClassifier(learning_rate=0.05, n_estimators=400, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb10 = XGBClassifier(learning_rate=0.1, n_estimators=400, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)
xgb11 = XGBClassifier(learning_rate=0.1, n_estimators=500, max_depth=7, min_child_weight=1, subsample=0.8, colsample_bytree=0.8)

pipeline.classifiers = [rf1, rf2, rf3, rf4, rf5, rf6,# rf7, rf8, rf9,
                        # svm1, svm2, svm3, svm6, svm7, svm8,
                        xgb1, xgb2, xgb3, xgb4, xgb5, xgb6, xgb7, xgb8, xgb9, xgb10, xgb11]
pipeline.fitted_classifiers = {}

In [5]:
pipeline.fit_classifiers()

INFO:utils.ml:Fitting classifiers...
INFO:utils.ml:Fitting classifier: RandomForestClassifier0
INFO:utils.ml:Top 10 features for RandomForestClassifier0: [('color_moments_hsv_S_min', 0.017865464923274402), ('lbp_rad2_bins16_1', 0.01710482240236057), ('color_moments_rgb_G_max', 0.01592751747898636), ('color_moments_lab_B_skew', 0.014984896284545041), ('color_moments_lab_A_mean', 0.014520689987923625), ('color_moments_lab_A_min', 0.014255859298818983), ('lbp_rad2_bins16_8', 0.012850291140871783), ('color_moments_lab_A_median', 0.01267530814412657), ('color_moments_hsv_H_max', 0.012369107300456332), ('lbp_rad3_bins24_1', 0.012254212310280344)]
INFO:utils.ml:Fitted classifier: RandomForestClassifier0; Done in 13.310807228088379 seconds
INFO:utils.ml:Fitting classifier: RandomForestClassifier1
INFO:utils.ml:Top 10 features for RandomForestClassifier1: [('color_moments_lab_A_mean', 0.020283105513643315), ('lbp_rad2_bins16_1', 0.019999064228677324), ('color_moments_hsv_S_min', 0.0192677230771

In [6]:
pipeline.predict_with_classifiers(VAL_PATH)

INFO:utils.ml:Predicting with classifiers on dataset: C:\Users\gimes\Src\repos\CADx-Project\dataset\binary\val


Processed 5/159 batches.
Processed 10/159 batches.
Processed 15/159 batches.
Processed 20/159 batches.
Processed 25/159 batches.
Processed 30/159 batches.
Processed 35/159 batches.
Processed 40/159 batches.
Processed 45/159 batches.
Processed 50/159 batches.
Processed 55/159 batches.
Processed 60/159 batches.
Processed 65/159 batches.
Processed 70/159 batches.
Processed 75/159 batches.
Processed 80/159 batches.
Processed 85/159 batches.
Processed 90/159 batches.
Processed 95/159 batches.
Processed 100/159 batches.
Processed 105/159 batches.
Processed 110/159 batches.
Processed 115/159 batches.
Processed 120/159 batches.
Processed 125/159 batches.
Processed 130/159 batches.
Processed 135/159 batches.
Processed 140/159 batches.
Processed 145/159 batches.
Processed 150/159 batches.
Processed 155/159 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier0
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier1


Processed 159/159 batches.


INFO:utils.ml:Predictions made with classifier: RandomForestClassifier2
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier3
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier4
INFO:utils.ml:Predictions made with classifier: RandomForestClassifier5
INFO:utils.ml:Predictions made with classifier: XGBClassifier6
INFO:utils.ml:Predictions made with classifier: XGBClassifier7
INFO:utils.ml:Predictions made with classifier: XGBClassifier8
INFO:utils.ml:Predictions made with classifier: XGBClassifier9
INFO:utils.ml:Predictions made with classifier: XGBClassifier10
INFO:utils.ml:Predictions made with classifier: XGBClassifier11
INFO:utils.ml:Predictions made with classifier: XGBClassifier12
INFO:utils.ml:Predictions made with classifier: XGBClassifier13
INFO:utils.ml:Predictions made with classifier: XGBClassifier14
INFO:utils.ml:Predictions made with classifier: XGBClassifier15
INFO:utils.ml:Predictions made with classifier: XGBClassifier16


{'GT': array([0, 0, 0, ..., 1, 1, 1]),
 'RandomForestClassifier0': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier1': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier2': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier3': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier4': array([0, 0, 0, ..., 0, 0, 1]),
 'RandomForestClassifier5': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier6': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier7': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier8': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier9': array([0, 0, 0, ..., 1, 0, 1]),
 'XGBClassifier10': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier11': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier12': array([0, 0, 1, ..., 1, 0, 1]),
 'XGBClassifier13': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier14': array([0, 0, 1, ..., 0, 0, 1]),
 'XGBClassifier15': array([0, 0, 0, ..., 0, 0, 1]),
 'XGBClassifier16': array([0, 0, 0, ..., 0, 0, 1])}

In [8]:
import pandas as pd

df = pd.DataFrame(data=pipeline.calculate_metrics())
df = df.T
df

INFO:utils.ml:Metrics for classifier GT: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'kappa': 1.0}
INFO:utils.ml:Metrics for classifier RandomForestClassifier0: {'accuracy': 0.8295574288724974, 'precision': 0.8295074374216937, 'recall': 0.8295013071614119, 'f1': 0.8295043146790723, 'kappa': 0.659008653029671}
INFO:utils.ml:Metrics for classifier RandomForestClassifier1: {'accuracy': 0.8274499473129611, 'precision': 0.8273946947310442, 'recall': 0.8274573315580559, 'f1': 0.8274163039735314, 'kappa': 0.654836656908431}
INFO:utils.ml:Metrics for classifier RandomForestClassifier2: {'accuracy': 0.8282402528977871, 'precision': 0.8281861154905702, 'recall': 0.8282524577827821, 'f1': 0.8282080157080156, 'kappa': 0.6564207056683409}
INFO:utils.ml:Metrics for classifier RandomForestClassifier3: {'accuracy': 0.8285036880927292, 'precision': 0.8285120982304082, 'recall': 0.8283647778658629, 'f1': 0.8284176564849545, 'kappa': 0.6568439142741609}
INFO:utils.ml:Metrics for classif

cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated with a value
cannot access local variable 'report' where it is not associated

,accuracy,precision,recall,f1,kappa
GT,1.000000,1.000000,1.000000,1.000000,1.000000
RandomForestClassifier0,0.829557,0.829507,0.829501,0.829504,0.659009
RandomForestClassifier1,0.827450,0.827395,0.827457,0.827416,0.654837
RandomForestClassifier2,0.828240,0.828186,0.828252,0.828208,0.656421
RandomForestClassifier3,0.828504,0.828512,0.828365,0.828418,0.656844
RandomForestClassifier4,0.828504,0.828512,0.828365,0.828418,0.656844
RandomForestClassifier5,0.831665,0.831642,0.831564,0.831596,0.663194
XGBClassifier6,0.829557,0.830716,0.829016,0.829203,0.658677
XGBClassifier7,0.830611,0.831998,0.830024,0.830217,0.660756
XGBClassifier8,0.831401,0.832912,0.830792,0.830987,0.662323


In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

highest = df[df["accuracy"] == df.iloc[1:]["accuracy"].max()]
idx = highest.index[0]

print(classification_report(pipeline.predictions["GT"], pipeline.predictions[idx]))
print(confusion_matrix(pipeline.predictions["GT"], pipeline.predictions[idx]))


              precision    recall  f1-score   support

           0       0.82      0.87      0.84      1931
           1       0.86      0.80      0.83      1865

    accuracy                           0.84      3796
   macro avg       0.84      0.84      0.84      3796
weighted avg       0.84      0.84      0.84      3796

[[1678  253]
 [ 371 1494]]
